# Practica 2 - Bag of Words
## Introducción a la Ciencia de datos

# Francisco Regalado

22 de septiembre del 2024

### Instrucciones

- Construir una representación basada en Bag of Words. Se deberá leer el archivo de texto (ver adjunto) que contiene múltiples entradas de texto y dos clases distintas. El vector de características final debe tener como última columna el atributo de clase.

Importar librerias necesarias


In [1]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

### Datos de Entrada

In [4]:
df = pd.read_csv('df_mini_HS.csv')
df

,label,text
0,1,ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1,1,"28: te amodio, odio a la perra de tu amiga ☺️☺..."
2,1,@LaDivinaDiva Callate maldita perra. O seguro ...
3,1,@MarysabelPuerto Mejor callate cara de puta o ...
4,1,@xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...
5,1,@CocotePR @ashleyhonohan callate puta
6,1,Y el inmigrante recibe ayuda del rico Estado l...
7,1,"De los moros no se puede esperar nada bueno, y..."
8,1,¿Por que si a una mujer le pegan un tiro en la...
9,1,"Analicemos esto: ¿Si te pones unos shorts así,..."


## Procesamiento



### Limpieza de Datos

Definimos una función para la limpieza de las oraciones contenidas en DF

In [9]:
def limpiar_txt(oracion):
    oracion = oracion.lower()  # Convertir a minúsculas
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub(r"http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar saltos de línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"[*]", "", oracion)  # Eliminar asteriscos
    oracion = re.sub(r"(\.)|(,)|(:)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de interrogación
    oracion = re.sub(r"[\"']", "", oracion)  # Eliminar comillas dobles y simples
    oracion = re.sub(r"(.)\1{2,}", r"\1", oracion)  # Reducir letras repetidas a una sola
    return oracion
    
def eliminar_emojis(oracion):
    # Expresión regular para emojis (rango Unicode para símbolos y emojis)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
        u"\U0001F680-\U0001F6FF"  # Transportes y símbolos adicionales
        u"\U0001F1E0-\U0001F1FF"  # Banderas (ISO country codes)
        u"\U00002702-\U000027B0"  # Otros símbolos y pictogramas
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    
    return emoji_pattern.sub(r'', oracion)  # Reemplaza emojis por una cadena vacía

In [11]:
# Aplicar la limpieza a la columna "texto"
df['text'] = df['text'].apply(limpiar_txt)
df['text'] = df['text'].apply(eliminar_emojis)

# Mostrar el DataFrame después de la limpieza
df

,label,text
0,1,esas cosas y otras pueden pasar por mantener a...
1,1,te amodio odio a la perra de tu amiga pero tú...
2,1,callate maldita perra o seguro eres un pobre ...
3,1,mejor callate cara de puta o reputa como tu m...
4,1,cállate puta
5,1,callate puta
6,1,y el inmigrante recibe ayuda del rico estado l...
7,1,de los moros no se puede esperar nada bueno y ...
8,1,por que si a una mujer le pegan un tiro en la ...
9,1,analicemos esto si te pones unos shorts así en...


### Tokenizacion

In [14]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/paco/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
# Definimos la funcion para tokenizar el data frame
def tokenizar_texto(oracion):
    return word_tokenize(oracion)

In [18]:
df_token = df.copy()
df_token['text'] = df_token['text'].apply(tokenizar_texto)
df_token

,label,text
0,1,"[esas, cosas, y, otras, pueden, pasar, por, ma..."
1,1,"[te, amodio, odio, a, la, perra, de, tu, amiga..."
2,1,"[callate, maldita, perra, o, seguro, eres, un,..."
3,1,"[mejor, callate, cara, de, puta, o, reputa, co..."
4,1,"[cállate, puta]"
5,1,"[callate, puta]"
6,1,"[y, el, inmigrante, recibe, ayuda, del, rico, ..."
7,1,"[de, los, moros, no, se, puede, esperar, nada,..."
8,1,"[por, que, si, a, una, mujer, le, pegan, un, t..."
9,1,"[analicemos, esto, si, te, pones, unos, shorts..."


### Stopwords

In [21]:
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /Users/paco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# Definimos la funcion para eliminar palabras vacias
def eliminar_stopwords(texto):
    # Filtrar las palabras que no son stopwords
    palabras_limpias = [texto for texto in texto if texto not in spanish_stopwords]
    return ' '.join(palabras_limpias)  # Unir las palabras de nuevo en una oración


In [25]:
df_token_stw = df_token.copy()
df_token_stw['text'] = df_token_stw['text'].apply(eliminar_stopwords)
df_token_stw

,label,text
0,1,cosas pueden pasar mantener inmigracion ilegal...
1,1,amodio odio perra amiga vida tijerazo
2,1,callate maldita perra seguro pobre marico detr...
3,1,mejor callate cara puta reputa madre nota hace...
4,1,cállate puta
5,1,callate puta
6,1,inmigrante recibe ayuda rico ladrón olvida nac...
7,1,moros puede esperar bueno dicen propios árabes...
8,1,si mujer pegan tiro cabeza dura tres dias mori...
9,1,analicemos si pones shorts así calle esperas d...


### Lematizacion

In [28]:
nlp = spc.load('es_core_news_sm')
# Función para lematizar el texto
def lematizar_texto(texto):
    doc = nlp(texto)  # Procesar el texto con spaCy
    # Extraer los lemas para cada token
    lemas = [token.lemma_ for token in doc]
    return " ".join(lemas)  # Unir los lemas en una cadena de texto

In [30]:
df_lema = df_token_stw.copy()
df_lema['text'] = df_lema['text'].apply(lematizar_texto)
df_lema

,label,text
0,1,cosa poder pasar mantener inmigracion ilegal e...
1,1,amodio odio perra amigo vida tijerazo
2,1,callate maldito perra seguro pobre marico detr...
3,1,mejor callate cara puta reputa madre nota hace...
4,1,cállate puto
5,1,callate puto
6,1,inmigrante recibir ayuda rico ladrón olvidar n...
7,1,moro poder esperar bueno decir propio árabe lu...
8,1,si mujer peguir tiro cabeza durar tres dia mor...
9,1,analicer si pón shorts así calle esperas decir...


### Bag of Words

In [33]:
# Instanciar el vectorizador
vectorizer = CountVectorizer()

# Aplicar Bag of Words a la columna 'text'
X = vectorizer.fit_transform(df_lema['text'])

# Convertir el resultado a un DataFrame con las palabras como columnas
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


# Convertir la columa "Label" de los datos originales a la columna "Clase". Ademas
# los 1 contenidos en esta columna representan si la oracion es sexista o no
df_lema.rename(columns={'label': 'Clase'}, inplace=True)
df_lema['Clase'] = df_lema['Clase'].replace(1, 'Sexismo')

# Agregamos la columna Clase a DataFrame del BagOfWords
df_bow['Clase'] = df_lema['Clase']
df_bow

,acoso,agar,agresión,amigo,amodio,analicer,aprieto,arab,arar,así,...,vez,vida,viola,voolka,vía,yogurín,you,árabe,él,Clase
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Sexismo
1,0,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Sexismo
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sexismo
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sexismo
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sexismo
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sexismo
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Sexismo
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Sexismo
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Sexismo
9,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Sexismo
